In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [6]:
file_path = "../data/extract_250m/final_score.csv"

In [7]:
df = pd.read_csv(file_path)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head(3)

,id,left,top,right,bottom,row_index,col_index,area,perimeter,geometry,land_score,dem_score,dni_score,temp_score,pvout_score,dso_score,pv_score
0,676612,414729.589868,369375.174135,414979.589868,369125.174135,306,855,6656.084138,410.853786,"MULTIPOLYGON (((17.7802286 51.1843321, 17.7801...",1.0,177.928571,1019.047974,9.4,1114.011963,0.034582,0.811841
1,676611,414729.589868,369625.174135,414979.589868,369375.174135,305,855,28107.499155,755.139192,"MULTIPOLYGON (((17.7818396 51.186814799999986,...",1.0,177.714286,1020.970893,9.4,1114.011963,0.033724,0.816066
2,676610,414729.589868,369875.174135,414979.589868,369625.174135,304,855,56502.279292,946.460361,"MULTIPOLYGON (((17.782959 51.188250499999995, ...",1.0,176.648936,1020.913394,9.4,1114.011963,0.032380,0.821032


In [8]:
cols_to_drop = ['left', 'top', 'right', 'bottom', 'row_index', 'col_index', 'geometry']
df_reduced = df.drop(columns=cols_to_drop)
print(df_reduced.shape)
df_reduced.head(3)

(321820, 10)


,id,area,perimeter,land_score,dem_score,dni_score,temp_score,pvout_score,dso_score,pv_score
0,676612,6656.084138,410.853786,1.0,177.928571,1019.047974,9.4,1114.011963,0.034582,0.811841
1,676611,28107.499155,755.139192,1.0,177.714286,1020.970893,9.4,1114.011963,0.033724,0.816066
2,676610,56502.279292,946.460361,1.0,176.648936,1020.913394,9.4,1114.011963,0.032380,0.821032


In [9]:
criteria = ['land_score','dem_score','dni_score','temp_score','pvout_score','dso_score','pv_score']
types    = np.array([+1,        -1,        +1,        +1,          +1,          +1,        +1], dtype=int)

# columns already scaled to [0,1] and benefit-type; leave them untouched
already_scaled = {'land_score', 'dso_score', 'pv_score'}

#build normalized matrix Xn (selective min–max)
X = df_reduced[criteria].to_numpy(float)
Xn = np.empty_like(X, float)

for j, colname in enumerate(criteria):
    col = X[:, j]
    if colname in already_scaled:
        Xn[:, j] = col  # keep as is
    else:
        cmin, cmax = np.nanmin(col), np.nanmax(col)
        if np.isclose(cmax, cmin):
            Xn[:, j] = 0.0
        else:
            Xn[:, j] = (col - cmin) / (cmax - cmin) if types[j]==+1 else (cmax - col) / (cmax - cmin)

# CRITIC weights
sigma = np.nanstd(Xn, axis=0, ddof=1)
R = np.corrcoef(Xn + 1e-12*np.random.randn(*Xn.shape), rowvar=False)
R = np.clip(R, -1.0, 1.0)
C = sigma * np.sum(1.0 - np.abs(R), axis=1)

w = np.ones_like(C)/len(C) if np.allclose(C.sum(), 0.0) else C / C.sum()

weights_df = pd.DataFrame({'Criterion': criteria, 'CRITIC_Weight': w}).sort_values('CRITIC_Weight', ascending=False)
display(weights_df.style.background_gradient(cmap='Blues').format({'CRITIC_Weight':'{:.4f}'}).set_caption("CRITIC Weights"))

/Users/sell/[1] Work/[0]PWR/[2] Project Work/Daina/PV_project/.venv_qgis/lib/python3.9/site-packages/matplotlib/projections/__init__.py:63: UserWarning:

Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.



,Criterion,CRITIC_Weight
0,land_score,0.3638
5,dso_score,0.1683
6,pv_score,0.1652
1,dem_score,0.0961
3,temp_score,0.0957
4,pvout_score,0.0663
2,dni_score,0.0447


In [10]:
# --- TOPSIS ranking (using pymcdm) ---
from pymcdm.methods import TOPSIS

# weights
w_vec = w

# Since Xn already has costs inverted, pass all +1 to TOPSIS
types_for_methods = np.ones(len(criteria), dtype=int)

topsis = TOPSIS()
topsis_scores = topsis(Xn, w_vec, types=types_for_methods)

# Ranks: 1 = best
topsis_ranks = (-topsis_scores).argsort().argsort() + 1

# Collect results and save
res_topsis = pd.DataFrame({
    "id": df_reduced["id"].values,
    "topsis_score": topsis_scores,
    "topsis_rank": topsis_ranks
})

out_path = "/content/drive/MyDrive/DAINA 3/mcdm_critic_topsis.csv"
res_topsis.to_csv(out_path, index=False)

# Quick look: top-10 cells
display(res_topsis.sort_values("topsis_rank").head(10))

# Optional: score distribution plot
fig = px.histogram(res_topsis, x="topsis_score", nbins=40, template="plotly_white",
                   title="TOPSIS Score Distribution")
fig.show()


AttributeError: module 'matplotlib.cbook' has no attribute '_rename_parameter'